In [79]:
from stormvogel import *
print("sanity")

sanity


In [89]:
%%prism grid_prism
mdp

const int N = 5; // Grid size

const int R_X = 0; // Ranger and lions initial positions
const int R_Y = N-1;
const int F_X = 0;
const int F_Y = 0;
const int C_X = N-1;
const int C_Y = (N-1)/2;


const int C1_X = 1; // Camera positions
const int C1_Y = 2;
const int C2_X = (N-1)/2;
const int C2_Y = 0;
const int C3_X = N-1;
const int C3_Y = N-2;

module ranger
    r_x : [0..N-1] init R_X;
    r_y : [0..N-1] init R_Y;
    
    //c_1 : [0..2] init 0; // camera 1, 0: nothing, 1: fang, 2: claw
    //c_2: [0..2] init 0;
    //c_3: [0..2] init 0;
    i_c: bool init false; // inspected claw
    i_f: bool init false; // inspected fang

    //decision : [0..5] init 0;
    // 0: No decision made
    // 1: left
    // 2: right
    // 3: up
    // 4: down
    // 5: inspect

    //[left] (r_x > 0) -> 1 : (decision' = 1);
    //[right] (r_x < N-1) -> 1 : (decision' = 2);
    //[up] (r_y > 0) -> 1 : (decision' = 3);
    //[down] (r_y < N-1) -> 1 : (decision' = 4);
    //[inspect] ( on_fang | on_claw | on_c1 | on_c1 | on_c2 ) -> 1 : (decision' = 5);

    //[step] (decision > 0) -> 1 : (decision' = 0);
    //
    //[step] (decision = 1) -> 0.8 : (r_x' = r_x - 1) & (r_y' = r_y)
    //                       + 0.2 : (r_x' = r_x) & (r_y' = r_y);
    //[step] (decision = 2) -> 0.8: (r_x' = r_x + 1) & (r_y' = r_y)
    //                       + 0.2 : (r_x' = r_x) & (r_y' = r_y);
    //[step] (decision = 3) -> 0.8: (r_x' = r_x) & (r_y' = r_y - 1)
    //                       + 0.2 : (r_x' = r_x) & (r_y' = r_y);
    //[step] (decision = 4) -> 0.8: (r_x' = r_x) & (r_y' = r_y + 1)
    //                       + 0.2 : (r_x' = r_x) & (r_y' = r_y);

    //[step] (decision = 5 & on_fang) -> 0.95 : (i_f' = true)
    //                                 + 0.05 : (i_f' = i_f);
    //[step] (decision = 5 & on_claw) -> 0.95 : (i_c' = true)
    //                                 + 0.05 : (i_c' = i_c);
    //[step] (decision = 5 & on_c1) -> 1 : (c_1' = 0);
    //[step] (decision = 5 & on_c2) -> 1 : (c_2' = 0);
    //[step] (decision = 5 & on_c3) -> 1 : (c_3' = 0);
    //[step] (decision = 5 & on_c1 & c_1 = 0) -> 1 : (i_f' = i_f);
    //[step] (decision = 5 & on_c1 & c_1 = 1) -> 0.6 : (i_f' = true)
    //                                        + 0.4 : (i_f' = i_f);
    //[step] (decision = 5 & on_c1 & c_1 = 2) -> 0.6 : (i_c' = true)
    //                                        + 0.4 : (i_c' = i_c);
                                                     
    //[step] (decision = 5 & on_c2 & c_2 = 0) -> 1 : (i_f' = i_f);
    //[step] (decision = 5 & on_c2 & c_2 = 1) -> 0.6 : (i_f' = true)
    //                                        + 0.4 : (i_f' = i_f);
    //[step] (decision = 5 & on_c2 & c_2 = 2) -> 0.6 : (i_c' = true)
    //                                        + 0.4 : (i_c' = i_c);
                                                     
    //[step] (decision = 5 & on_c3 & c_3 = 0) -> 1 : (i_f' = i_f);
    //[step] (decision = 5 & on_c3 & c_3 = 1) -> 0.6 : (i_f' = true)
    //                                        + 0.4 : (i_f' = i_f);
    //[step] (decision = 5 & on_c3 & c_3 = 2) -> 0.6 : (i_c' = true)
    //                                        + 0.4 : (i_c' = i_c);
    //[step] (c_x = C1_X & c_y = C1_Y) -> 1 : (c_1' = 1);
    //[step] (c_x = C2_X & c_y = C2_Y) -> 1 : (c_2' = 1);
    //[step] (c_x = C3_X & c_y = C3_Y) -> 1 : (c_3' = 1);
endmodule

formula on_fang = r_x = f_x & r_y = f_y;
formula on_claw = r_x = c_x & r_y = c_y;
formula on_c1 = r_x = C1_X & r_y = C1_Y;
formula on_c2 = r_x = C2_X & r_y = C2_Y;
formula on_c3 = r_x = C3_X & r_y = C3_Y;

module fang
    f_x : [0..N-1] init F_X;
    f_y : [0..N-1] init F_Y;

    [step] (fang_move_left = 1)  -> 0.12 / fang_moves : (f_x' = f_x - 1) & (f_y' = f_y);
    [step] (fang_move_right = 1) -> 0.12 / fang_moves : (f_x' = f_x + 1) & (f_y' = f_y);
    [step] (fang_move_up = 1) -> 0.12 / fang_moves : (f_x' = f_x) & (f_y' = f_y - 1);
    [step] (fang_move_down = 1) -> 0.12 / fang_moves : (f_x' = f_x) & (f_y' = f_y + 1);
endmodule

module claw
    c_x : [0..N-1] init C_X; // Claw tribe
    c_y : [0..N-1] init C_Y;

    [step] (claw_move_left = 1)  -> 0.12 / claw_moves : (c_x' = c_x - 1) & (c_y' = c_y);
    [step] (claw_move_right = 1) -> 0.12 / claw_moves : (c_x' = c_x + 1) & (c_y' = c_y);
    [step] (claw_move_up = 1) -> 0.12 / claw_moves : (c_x' = c_x) & (c_y' = c_y - 1);
    [step] (claw_move_down = 1) -> 0.12 / claw_moves : (c_x' = c_x) & (c_y' = c_y + 1);
endmodule

formula tribe_distance_x = f_x - c_x;
formula tribe_distance_y = f_y - c_y;

// Lion movement
formula lion_move_h1 = ((pow(pow(tribe_distance_y,2),0.5) > 1) | 
                        (tribe_distance_x > 2) | 
                        (tribe_distance_x > 1 & (pow(pow(tribe_distance_y,2),0.5) = 1)) |
                        (tribe_distance_x = 1 & (pow(pow(tribe_distance_y,2),0.5) > 1)));

formula lion_move_h2 = ((pow(pow(tribe_distance_y,2),0.5) > 1) | 
                       (tribe_distance_x < -2) | 
                       (tribe_distance_x < -1 & (pow(pow(tribe_distance_y,2),0.5) = 1)) |
                       (tribe_distance_x = -1 & (pow(pow(tribe_distance_y,2),0.5) > 1)));

formula lion_move_v1 = ((pow(pow(tribe_distance_x,2),0.5) > 1) | 
                        (tribe_distance_y > 2) | 
                        (tribe_distance_y > 1 & (pow(pow(tribe_distance_x,2),0.5) = 1)) |
                        (tribe_distance_y = 1 & (pow(pow(tribe_distance_x,2),0.5) > 1)));

formula lion_move_v2 = ((pow(pow(tribe_distance_x,2),0.5) > 1) | 
                        (tribe_distance_y < -2) | 
                        (tribe_distance_y < -1 & (pow(pow(tribe_distance_x,2),0.5) = 1)) |
                        (tribe_distance_y = -1 & (pow(pow(tribe_distance_x,2),0.5) > 1)));

// Fang movement possibilities
formula fang_move_left  = ((f_x > 0) & lion_move_h1) ? 1 : 0;
formula fang_move_right = ((f_x < N-1) & lion_move_h2) ? 1 : 0;
formula fang_move_up    = ((f_y > 0) & lion_move_v1) ? 1 : 0;
formula fang_move_down  = ((f_y < N-1) & lion_move_v2) ? 1 : 0;
formula fang_moves = fang_move_left + fang_move_right + fang_move_up + fang_move_down;

// Claw movement possibilities
formula claw_move_left  = ((c_x > 0) & lion_move_h2) ? 1 : 0;
formula claw_move_right = ((c_x < N-1) & lion_move_h1) ? 1 : 0;
formula claw_move_up    = ((c_y > 0) & lion_move_v2) ? 1 : 0;
formula claw_move_down  = ((c_y < N-1) & lion_move_v1) ? 1 : 0;
formula claw_moves = claw_move_left + claw_move_right + claw_move_up + claw_move_down;

rewards "steps"
    [step] true : 1; //whenever a step is taken, add 1 to the timesteps reward.
endrewards

// assign labels to inspected fang/claw states.
label "inspected_fang" = i_f;
label "inspected_claw" = i_c;
label "inspected_both" = i_f & i_c;
label "overlap" = f_x = c_x & f_y = c_y;

// -- LTLSPEC --

// Tribes never in same sector
// "Pmax=? [G !\"overlap\"]"

// Best effort prob that health of entire population is estimated
// "Pmax=? [F{\"steps\"}<=14 \"inspected_both\"]

// Which tribe is easiest to go after first?
// "multi(R{\"steps\"}min? [F{\"steps\"}<=14 \"inspected_claw\"], R{\"steps\"}min? [F{\"steps\"}<=14 \"inspected_fang\"])"

In [101]:
import stormpy
model = stormpy.build_model(grid_prism)
print(model)
properties = stormpy.parse_properties("Pmax=? [G ! \"overlap\"]")
result = stormpy.model_checking(model, properties[0])
print("RESULT:")
print(result.at(model.initial_states[0]))

no_transitions = model.transition_matrix.nr_rows
if no_transitions < 100:
    sv_model = mapping.stormpy_to_stormvogel(model)
    show(sv_model, show_editor=True)

-------------------------------------------------------------- 
Model type: 	MDP (sparse)
States: 	280
Transitions: 	1556
Choices: 	1556
Reward Models:  steps
State Labels: 	6 labels
   * deadlock -> 56 item(s)
   * init -> 1 item(s)
   * overlap -> 0 item(s)
   * inspected_fang -> 0 item(s)
   * inspected_claw -> 0 item(s)
   * inspected_both -> 0 item(s)
Choice Labels: 	none
-------------------------------------------------------------- 

RESULT:
1.0
